In [1]:
import argparse
import yaml
import torch
from torchsummary import summary
from thop import profile, clever_format

def import_class(name):
    components = name.split('.')
    mod = __import__(components[0])  # import return model
    for comp in components[1:]:
        mod = getattr(mod, comp)
    return mod

In [2]:
%cd ..

d:\Code\DATN\project\Pose-based-WLASL


## Count Params and MACs 

In [3]:
# Load config
with open(r'.\configs\ctr-gcn\config.yaml') as f: 
    arg = yaml.load(f, Loader=yaml.FullLoader)
arg = argparse.Namespace(**arg)

num_keypoint = 27
T_frame = 150
arg.model_args['num_class'] = 2000
arg.model_args['graph_args']['layout'] = f'keypoint-{num_keypoint}'
vars(arg)

{'Experiment_name': 'ctr-gcn',
 'phase': 'train',
 'print_log': True,
 'save_interval': 1,
 'eval_interval': 1,
 'show_topk': [1],
 'num_worker': 2,
 'model_saved_dir': './save_models/',
 'work_dir': './work_dir/',
 'use_nla': False,
 'label_smoothing': 0,
 'word_embedding': 'D:\\DATN\\project\\data\\preprocessed_data\\keypoints_NLA_WLASL\\wlasl_word_embeddings.pkl',
 'vocab': 'D:\\DATN\\project\\data\\preprocessed_data\\wlasl_class_list.txt',
 'feeder': 'feeder.feeder.Feeder',
 'train_feeder_args': {'data_path': 'D:\\DATN\\project\\data\\preprocessed_data\\skeleton\\WLASL100\\train_data_joint.npy',
  'label_path': 'D:\\DATN\\project\\data\\preprocessed_data\\skeleton\\WLASL100\\train_label.pkl',
  'random_choose': True,
  'random_move': True,
  'window_size': 80,
  'random_shift': True,
  'normalization': True,
  'random_mirror': True,
  'random_mirror_p': 0.5,
  'is_vector': False,
  'max_xy': 256},
 'test_feeder_args': {'data_path': 'D:\\DATN\\project\\data\\preprocessed_data\\skele

In [4]:
# Load model
Model = import_class(arg.model)
model = Model(**arg.model_args)

# Input size: (N, C, T, V, M)
# Batch size N = 64
# Channel C = 3 (stand for (X,Y,C) skeleton point data) 
# T = 150 (number of frames)
# V = 27 (number of keypoints)
# M = 1 (number of persons)

summary(model, input_size=(3, T_frame, num_keypoint, 1))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm1d-1              [-1, 81, 150]             162
            Conv2d-2           [-1, 8, 150, 27]              32
            Conv2d-3           [-1, 8, 150, 27]              32
            Conv2d-4          [-1, 64, 150, 27]             256
              Tanh-5            [-1, 8, 27, 27]               0
            Conv2d-6           [-1, 64, 27, 27]             576
             CTRGC-7          [-1, 64, 150, 27]               0
            Conv2d-8           [-1, 8, 150, 27]              32
            Conv2d-9           [-1, 8, 150, 27]              32
           Conv2d-10          [-1, 64, 150, 27]             256
             Tanh-11            [-1, 8, 27, 27]               0
           Conv2d-12           [-1, 64, 27, 27]             576
            CTRGC-13          [-1, 64, 150, 27]               0
           Conv2d-14           [-1, 8, 

In [5]:
input = torch.randn(1, 3, T_frame, num_keypoint, 1)
macs, params = profile(model, inputs=(input, ))
macs, params = clever_format([macs, params], "%.3f")
print("MACs: {}, Params: {}".format(macs, params))

[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_softmax() for <class 'torch.nn.modules.activation.Softmax'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
MACs: 2.201G, Params: 1.926M
